# はじめに


このKernelは、**Happywhale - Whale and Dolphin Identification competition** のデータセットを調査します。

<img src="https://images.unsplash.com/photo-1570913179118-f3d24be1d1f7?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=2143&q=80" width=500></img>

# 分析の準備

データをロードして、事前調査をしてみましょう。

<img src="https://images.unsplash.com/photo-1568430328012-21ed450453ea?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1174&q=80" width=500></img>

In [ ]:
!pip install imagesize

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import imagesize

In [ ]:
print(f"Files and folders: {os.listdir('/kaggle/input/happy-whale-and-dolphin')}")

まず、`train.csv` と `sample_submission.csv` をロードしてみましょう。

In [ ]:
train_df = pd.read_csv('/kaggle/input/happy-whale-and-dolphin/train.csv')
submission_df = pd.read_csv('/kaggle/input/happy-whale-and-dolphin/sample_submission.csv')

In [ ]:
train_df.head()

In [ ]:
submission_df.head()

# データ探索

<img src="https://images.unsplash.com/photo-1611890129309-31e797820019?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1170&q=80" width=500></img>

訓練データと訓練画像、テスト画像について、もう少し詳しく見てみましょう。

In [ ]:
print(f"Images in train index file: {train_df.image.nunique()}")
print(f"Species in train index file: {train_df.species.nunique()}")
print(f"Individual IDs in train index file: {train_df.individual_id.nunique()}")

print(f"Images in train images folder: {len(os.listdir('/kaggle/input/happy-whale-and-dolphin/train_images'))}")
print(f"Images in test images folder: {len(os.listdir('/kaggle/input/happy-whale-and-dolphin/test_images'))}")

全種類に目を向けてみよう。

In [ ]:
print(f"Species: {train_df.species.unique()}")

このコンペの掲示板や他のノートブック（ex: [Happywhale: Data Distribution](https://www.kaggle.com/awsaf49/happywhale-data-distribution/notebook)）での議論から、次のようなことがわかります。
* シロイルカとグロビスはクジラである。 
* イルカとクジラの区別がつくので、ベルーガとグロバスの名前を変更する。 
また、次のことも観察されます。
* バンドウイルカはタイプミス（dolpin）です。
* 殺人鯨は`kiler`と間違って入力されています。


In [ ]:
train_df.loc[train_df.species.str.contains('beluga'), 'species'] = 'beluga_whale'
train_df.loc[train_df.species.str.contains('globis'), 'species'] = 'globis_whale'

In [ ]:
train_df['class'] = train_df.species.map(lambda x: 'whale' if 'whale' in x else 'dolphin')

In [ ]:
train_df['species'] = train_df['species'].str.replace('bottlenose_dolpin','bottlenose_dolphin')
train_df['species'] = train_df['species'].str.replace('kiler_whale','killer_whale')

イルカVSクジラって何種類あるのか確認してみよう。

In [ ]:
temp = train_df.groupby(["class"])["species"].nunique()
df = pd.DataFrame({'Classes': temp.index,
                   'Species': temp.values
                  })
df = df.sort_values(['Species'], ascending=False)
plt.figure(figsize = (6,6))
plt.title('Species distribution - grouped on Dolphins and Whales - train dataset')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Classes', y="Species", data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

`train_df` の値分布から `individual_id` というカラムの詳細を確認してみましょう。

In [ ]:
print("Top 10 individual_id")
train_df.individual_id.value_counts().head(10)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
sns.kdeplot(np.log(train_df.individual_id.value_counts()))
plt.title("Logaritmic distribution of individual_id frequency in images")
plt.show()

また、イルカとクジラは分けて考えましょう。

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
sns.kdeplot(np.log(train_df.loc[train_df["class"]=='whale'].individual_id.value_counts()))
sns.kdeplot(np.log(train_df.loc[train_df["class"]=='dolphin'].individual_id.value_counts()))
ax.legend(labels=['whale', 'dolphin'])
plt.title("Logaritmic distribution of individual_id frequency in images")
plt.show()

訓練データセットに含まれる種の頻度も確認してみましょう。

In [ ]:
df = train_df.groupby(["class", "species"])["image"].count().reset_index()
df.columns = ["Class", "Species", "Images"]
df = df.sort_values(['Images'], ascending=False)
plt.figure(figsize = (12,6))
plt.title('Species distribution - images per each species - train dataset')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Species', y="Images", hue='Class', data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

では、それぞれの種にいくつの個体IDがあるのか見てみましょう。

In [ ]:
df = train_df.groupby(["class", "species"])["individual_id"].nunique().reset_index()
df.columns = ["Class", "Species", "Unique ID Count"]
df = df.sort_values(["Unique ID Count"], ascending=False)

df = df.sort_values(['Unique ID Count'], ascending=False)
plt.figure(figsize = (12,6))
plt.title('Species distribution - Individual IDs per each species - train dataset')
sns.set_color_codes("pastel")
s = sns.barplot(x = 'Species', y="Unique ID Count", hue='Class', data=df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

それでは、トレーニング画像とテスト画像のデータセットで、画像サイズを確認してみましょう。

`train_df` にリストされたイメージのセットが、 `train_images` フォルダにあるイメージのセットと同じかどうか確認してみましょう。

In [ ]:
train_df_list = list(train_df.image.unique())
train_images_list = list(os.listdir('/kaggle/input/happy-whale-and-dolphin/train_images'))
delta = set(train_df_list) & set(train_images_list)
minus = set(train_df_list) - set(train_images_list)
print(f"Images in train dataset: {len(train_df_list)}\nImages in train folder: {len(train_images_list)}\nIntersection: {len(delta)}\nDifference: {len(minus)}")

`train_df` にインデックスされたすべての画像は images フォルダに存在し、その逆も同様です。

# 画像データ探索

まず、どちらの関数（cv2ベースとimagesizeベース）が速く実行されるかをテストします。

In [ ]:
# image size using cv2 imread shape
def read_image_sizes_cv2(file_name):
    image = cv2.imread('/kaggle/input/happy-whale-and-dolphin/train_images/' + file_name)
    return list(image.shape)

In [ ]:
# image size using imagesize
def get_image_sizes_imagesize(file_name):
    width, height = imagesize.get('/kaggle/input/happy-whale-and-dolphin/train_images/' + file_name)
    return [width, height]

In [ ]:
import time
sample_size = 100
start_time = time.time()
train_sample_df = train_df.sample(sample_size)
m = np.stack(train_sample_df['image'].apply(read_image_sizes_cv2))
df = pd.DataFrame(m,columns=['w','h','c'])
print(f"Total processing time for {sample_size} images (using cv2): {round(time.time()-start_time, 2)} sec.")

In [ ]:
import time
sample_size = 100
start_time = time.time()
train_sample_df = train_df.sample(sample_size)
m = np.stack(train_sample_df['image'].apply(get_image_sizes_imagesize))
df = pd.DataFrame(m,columns=['w','h'])
print(f"Total processing time for {sample_size} images (using imagesize): {round(time.time()-start_time, 2)} sec.")

この関数は、より効果的であるため、イメージサイズベースの関数を使用することにしました。
2500サンプルで実行する。

In [ ]:
import time
sample_size = 2500
start_time = time.time()
train_sample_df = train_df.sample(sample_size)
m = np.stack(train_sample_df['image'].apply(get_image_sizes_imagesize))
df = pd.DataFrame(m,columns=['w','h'])
print(f"Total processing time for {sample_size} images (using imagesize): {round(time.time()-start_time, 2)} sec.")

In [ ]:
train_img_df = pd.concat([train_sample_df, df], axis=1, sort=False)
print(f"Number of different image size ( images samples): {train_img_df.groupby(['w','h']).count().shape[0]}")

画像サイズが多いようです（全画像数の5％弱しかサンプリングしていません）。

種ごとの幅・高さ、色の分布を可視化してみましょう。

In [ ]:
plt.figure(figsize = (12,6))
plt.title('Species distribution - width per each species - train dataset (5% random data sample)')
sns.set_color_codes("pastel")
s = sns.boxplot(x = 'species', y="w", data=train_img_df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

In [ ]:
plt.figure(figsize = (12,6))
plt.title('Species distribution - height per each species - train dataset (5% random data sample)')
sns.set_color_codes("pastel")
s = sns.boxplot(x = 'species', y="h", data=train_img_df)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
locs, labels = plt.xticks()
plt.show()

種ごとの幅と高さの分布を散布図を使って示してみよう。

In [ ]:
def plot_species_scatter(train_img_df):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    species = list(train_img_df.species.unique())
    fig, ax = plt.subplots(5, 5,figsize=(15, 12))

    for spec in species:
        i += 1
        plt.subplot(5, 5,i)
        df = train_img_df.loc[train_img_df.species==spec]
        plt.scatter(df['w'], df['h'], marker='+')
        plt.xlabel(spec, fontsize=9)
    plt.show();
plot_species_scatter(train_img_df.dropna())

5%のランダムサンプルでは、色数は常に3色であるようです。

種族ごとにグループ化された訓練画像のいくつかをサンプルにしてみましょう。 

まず、プロット関数を作成します。

In [ ]:
def plot_image_samples(species):
    root_path = "/kaggle/input/happy-whale-and-dolphin/"
    fig.subplots_adjust(hspace = .1, wspace=.1)
    images_folder="train_images/"
    df = train_df[train_df['species']==species].copy()
    df.index = range(len(df.index))

    f, ax = plt.subplots(4, 4, figsize=(16,16))

    for i in range(16):
        file = df.loc[i, 'image']
        species = df.loc[i, 'species']
        identifier = df.loc[i, 'individual_id']
        img = cv2.imread(root_path+images_folder+file)
        ax[i//4, i%4].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax[i//4, i%4].set_title(identifier+" ("+species+")")
        ax[i//4, i%4].axis('off')

In [ ]:
plot_image_samples("bottlenose_dolphin")

In [ ]:
plot_image_samples("beluga_whale")

In [ ]:
plot_image_samples("humpback_whale")

In [ ]:
plot_image_samples("blue_whale")

In [ ]:
plot_image_samples("killer_whale")

In [ ]:
plot_image_samples("spotted_dolphin")

テスト画像のサンプルも見てみましょう。

In [ ]:
def plot_image_samples_test():
    root_path = "/kaggle/input/happy-whale-and-dolphin/"
    fig.subplots_adjust(hspace = .1, wspace=.1)
    images_folder="test_images/"

    f, ax = plt.subplots(4, 4, figsize=(16,16))
    file_list = list(os.listdir(root_path+images_folder))
    for i in range(16):
        file = file_list[i]
        img = cv2.imread(root_path+images_folder+file)
        ax[i//4, i%4].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax[i//4, i%4].set_title("Test image: "+file)
        ax[i//4, i%4].axis('off')

In [ ]:
plot_image_samples_test()

# 提出

<img src="https://images.unsplash.com/photo-1602264985195-52b338cb937b?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1170&q=80" width=500></img>

識別子を回転させて、`new_individual`が最初の選択肢になるようにしましょう。

In [ ]:
def rotate_values(x):
    xcopy = x.split()
    temp = xcopy[4]
    xcopy[4] = xcopy[0]
    xcopy[0] = temp
    xcopy = " ".join(xcopy)
    return xcopy

In [ ]:
submission_df["predictions"] = submission_df["predictions"].apply(lambda x: rotate_values(x))

In [ ]:
submission_df.head()

用意した投稿ファイルを出力します。

In [ ]:
submission_df.to_csv('submission.csv', index=False)